In [1]:
!pip install git+https://github.com/CSIRO-enviro-informatics/loci-scripts.git@master
!pip install numpy pandas

  Cloning https://github.com/CSIRO-enviro-informatics/loci-scripts.git (to revision master) to /tmp/pip-req-build-i779ebw4
  Running command git clone -q https://github.com/CSIRO-enviro-informatics/loci-scripts.git /tmp/pip-req-build-i779ebw4
  Running command git submodule update --init --recursive -q
  Created wheel for pyloci: filename=pyloci-0.1.1.25-py3-none-any.whl size=34955 sha256=48cae197cde659b6894eb8352e92a80d8ee0d17a803924fde1a0ec25b81d98f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-3gqp1m1r/wheels/3e/5b/79/764b8572d0ff50a032783c6b4b6373567cf1ca9264f188d83d
Successfully built pyloci


# Reapportioning ASGS16 LGA to SA1  example using the pyloci API

This notebook show how you would perform reapportioning using the [pyloci](https://pypi.org/project/pyloci/) library using a simple CSV file. The pyloci library interfaces directly with the SPARQL API for the Loc-I Cache GraphDB.

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('loci-lga-sa1-test-case1.csv', delimiter = ',')
#show the csv file read in
df

,LGA,Estimated Agricultural Area,Estimated value 2016
0,http://linked.data.gov.au/dataset/asgs2016/loc...,608800,118073000
1,http://linked.data.gov.au/dataset/asgs2016/loc...,144111000,39100
2,http://linked.data.gov.au/dataset/asgs2016/loc...,254500,142090000
3,http://linked.data.gov.au/dataset/asgs2016/loc...,18900,5195000
4,http://linked.data.gov.au/dataset/asgs2016/loc...,74100,55850000


In [3]:
#sure the config for the SPARQL endpoint to hit is set
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
print(os.getenv("SPARQL_ENDPOINT"))

http://db.loci.cat/repositories/loci-cache


In [4]:
#import the reapportioning module from pyloci
from pyloci import reapportioning


from pyloci.api.util import Util as API_Util
from pyloci.sparql import util as sparql_util
import time

auth = None
api_util = API_Util()


In [5]:
toFeatureType = "http://linked.data.gov.au/def/asgs#StatisticalAreaLevel1"
LOCI_INTEGRATION_API = "https://test-api.loci.cat/api/v1"

In [6]:
%%time
target_col = "Estimated value 2016"
res_df_cols = {
    "LGA" : [],
    "SA1" : [],
    "percent_overlap": [],
    target_col: [],
    "reapportioned_data": [],
}
for index, row in df.iterrows():
    fromFeature = row['LGA']
    print("{}, {}".format(row['LGA'], str(row[target_col])))
    print("Querying overlaps of {} to {}...".format(fromFeature, toFeatureType))
    tic = time.perf_counter()
    list_locations = api_util.query_api_location_overlaps(fromFeature, toFeatureType, LOCI_INTEGRATION_API, crosswalk='true')
    toc = time.perf_counter()
    print(f"query_api_location_overlaps took {toc - tic:0.4f} seconds")
    #print(list_locations)
    for o in list_locations['overlaps']:
        res_df_cols["LGA"].append(fromFeature)
        res_df_cols["SA1"].append(o['uri'])
        res_df_cols["percent_overlap"].append(o['forwardPercentage'])
        res_df_cols[target_col].append(row[target_col])
        reapportioned = (float(o['forwardPercentage'])/100.0)*float(row[target_col])
        res_df_cols["reapportioned_data"].append(reapportioned)

        

http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/55880, 118073000
Querying overlaps of http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/55880 to http://linked.data.gov.au/def/asgs#StatisticalAreaLevel1...
query_api_location_overlaps took 2.4012 seconds
http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/40070, 39100
Querying overlaps of http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/40070 to http://linked.data.gov.au/def/asgs#StatisticalAreaLevel1...
query_api_location_overlaps took 10.0137 seconds
http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/50080, 142090000
Querying overlaps of http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/50080 to http://linked.data.gov.au/def/asgs#StatisticalAreaLevel1...
query_api_location_overlaps took 11.1907 seconds
http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/10050, 5195000
Querying overlaps of http://linked.data.gov.au/dataset/asgs2016/localgovernmentar

In [7]:
res_df = pd.DataFrame (res_df_cols, columns = ['LGA', "SA1", "percent_overlap", target_col, "reapportioned_data"])
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', 0)
res_df

,LGA,SA1,percent_overlap,Estimated value 2016,reapportioned_data
0,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/55880,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/50902124207,20.17,118073000,23819723.86
1,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/55880,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/50902124212,79.83,118073000,94253277.73
2,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/40070,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40101100114,0.54,39100,210.65
3,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/40070,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40101100101,0.94,39100,368.57
4,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/40070,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40101100145,0.49,39100,190.31
...,...,...,...,...,...
328,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20110,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/20403106910,0.01,55850000,5442.51
329,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20110,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/20403106917,0.01,55850000,5251.61
330,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20110,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/20403107109,0.05,55850000,25943.50
331,http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/20110,http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/20403106920,0.01,55850000,7163.15
